In [7]:
%%capture
!pip install llama-index
!pip install llama-index-evaluation
!pip install llama-index-node_parser
!pip install llama-index-embeddings-langchain
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install -q transformers einops accelerate langchain bitsandbytes

In [25]:
import json, nest_asyncio, openai, os
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, Document
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.openai import OpenAI
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

nest_asyncio.apply()
api_keys = []
current_key_index = 2
openai.api_key = api_keys[current_key_index]

In [26]:
# load data
with open("/content/pos_json.json", "r", encoding="utf-8") as file:
    pos_data = json.load(file)

documents = [Document(text=entry['query'], doc_id=str(entry['id'])) for entry in pos_data]

In [27]:
llm = OpenAI(model="gpt-3.5-turbo")
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
)
vector_index = VectorStoreIndex(nodes, service_context=service_context)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<ipython-input-27-2844bc043789>:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
prompt_template = """
You are an assistant specialized in choosing alternative project. Review the following project description:
"{query}"
Identify the most appropriate project to replace the current one based on the description provided. Return only the name of that project.
"""

with open("/content/neg_json.json", "r", encoding="utf-8") as file:
    neg_data = json.load(file)

for id in range(len(neg_data)):
    entry = neg_data[id]
    prompt = prompt_template.format(query=entry["query"])
    query_engine = vector_index.as_query_engine()
    response = query_engine.query(prompt).response.strip()
    entry['alternative_method'] = response



with open("/content/neg_json_output.json", "w", encoding="utf-8") as file:
    json.dump(neg_data, file, indent=4)

print("Data saved back to neg_json_output.json")

from google.colab import files
file_path = "/content/neg_json_output.json"
files.download(file_path)